<a href="https://colab.research.google.com/github/filipemansano-mongodb/mongodb-rag-chatbot/blob/main/RAG_With_LlamaIndex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install llama-index
!pip install llama-index-vector-stores-mongodb
!pip install llama-index-embeddings-openai
!pip install pymongo

In [ ]:
import json
from llama_index.core import Document
from llama_index.core.schema import MetadataMode
import json

# sample
#{
#  "description": "Paciente queixa-se de dor de cabeça persistente e tontura há dois dias. Não há febre, mas tem dificuldade para dormir.",
#  "date": "12/04/2024 10:15:00",
#  "patient_name": "Marcia Silva",
#  "patient_age": 28,
#  "sedentary": "Não",
#  "clinic_id": 1,
#  "symptoms": "Dor de cabeça, tontura, insônia",
#  "medicine_prescriptions": "Paracetamol 500mg, Dramin"
#},

filepath = "/content/sample_data/patients.json";

with open(filepath, 'r') as f:
    data = json.load(f)

llama_documents = [];

for document in data:

  # Value for metadata fields must be one of (str, int, float, None)
  llama_document = Document(
    text=document["description"],
    metadata=document,
    excluded_llm_metadata_keys=["description", "clinic_id"],
    excluded_embed_metadata_keys=["description", "sedentary", "clinic_id"],
    metadata_template="{key}=>{value}",
    text_template="Metadata: {metadata_str}\n-----\nContent: {content}",
  )

  llama_documents.append(llama_document)

print("LLM Metada\n", llama_documents[0].get_content(metadata_mode=MetadataMode.LLM))
print("\n\nEmbed Metada\n", llama_documents[0].get_content(metadata_mode=MetadataMode.EMBED))

LLM Metada
 Metadata: date=>12/04/2024 10:15:00
patient_name=>Marcia Silva
patient_age=>28
sedentary=>Não
symptoms=>Dor de cabeça, tontura, insônia
medicine_prescriptions=>Paracetamol 500mg, Dramin
-----
Content: Paciente queixa-se de dor de cabeça persistente e tontura há dois dias. Não há febre, mas tem dificuldade para dormir.


Embed Metada
 Metadata: date=>12/04/2024 10:15:00
patient_name=>Marcia Silva
patient_age=>28
symptoms=>Dor de cabeça, tontura, insônia
medicine_prescriptions=>Paracetamol 500mg, Dramin
-----
Content: Paciente queixa-se de dor de cabeça persistente e tontura há dois dias. Não há febre, mas tem dificuldade para dormir.


In [ ]:
# configuring openai model
from llama_index.core.settings import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get('openai_api_key')

embed_model=OpenAIEmbedding(
  model="text-embedding-3-small",
  dimensions=256
)

llm=OpenAI()
Settings.llm=llm
Settings.embed_model=embed_model

In [ ]:
# creating nodes from LLhamaIndex Documents
from llama_index.core.node_parser import SentenceSplitter

parser = SentenceSplitter()
nodes = parser.get_nodes_from_documents(llama_documents)

for node in nodes:
    node_embedding = embed_model.get_text_embedding(
        node.get_content(metadata_mode="all")
    )
    node.embedding = node_embedding

In [ ]:
# creating mongodb connection
import pymongo
from google.colab import userdata

def get_mongo_client(mongo_uri):
  """Establish connection to the MongoDB."""
  try:
    client = pymongo.MongoClient(mongo_uri)
    print("Connection to MongoDB successful")
    return client
  except pymongo.errors.ConnectionFailure as e:
    print(f"Connection failed: {e}")
    return None

mongo_uri = userdata.get('MONGO_URI')
if not mongo_uri:
  print("MONGO_URI not set in environment variables")

mongo_client = get_mongo_client(mongo_uri)

Connection to MongoDB successful


In [ ]:
# create vector search index
index = {
  "definition": {
    "fields": [
          {
              "numDimensions": 256,
              "path": "embedding",
              "similarity": "cosine",
              "type": "vector"
          },
          {
              "path": "metadata.clinic_id",
              "type": "filter"
          }
      ],
  },
  "name": "vector_index",
  "type": "vectorSearch",
}

mongo_client.test.command({
    "createSearchIndexes": "rag",
    "indexes": [index]
  }
)

{'indexesCreated': [{'id': '661eac0a27b98a1249aecce3',
   'name': 'vector_index'}],
 'ok': 1.0,
 '$clusterTime': {'clusterTime': Timestamp(1713286154, 1),
  'signature': {'hash': b't\xc1\x01\xc5\x87\xe7P\x1aSo\xfcg\xc5\x92 \xadVhg\x80',
   'keyId': 7339894402132738054}},
 'operationTime': Timestamp(1713286154, 1)}

In [ ]:
# clear all results
mongo_client.test.rag.delete_many({})

DeleteResult({'n': 0, 'electionId': ObjectId('7fffffff000000000000001b'), 'opTime': {'ts': Timestamp(1713286233, 1), 't': 27}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1713286233, 1), 'signature': {'hash': b':\xe7\xe2\xab\x92M\xe2vt\x08\xd6\xc8p\x119\xf8\x17_S\xf5', 'keyId': 7339894402132738054}}, 'operationTime': Timestamp(1713286233, 1)}, acknowledged=True)

In [ ]:
from llama_index.vector_stores.mongodb import MongoDBAtlasVectorSearch

vector_store = MongoDBAtlasVectorSearch(
  mongo_client,
  db_name="test",
  collection_name="rag",
  index_name="vector_index"
)

vector_store.add(nodes)

['67e8a0c1-2b70-407c-974f-c7c38e45b9fe',
 '6ebf1320-8d0d-4789-bc49-b91777700da1',
 '10761f2e-40ce-4b0d-ae23-3e62484a4395',
 '00130cde-89ae-4245-9c26-310fdb4fd603',
 '424d7c77-7903-4470-9678-b42accc3a4a6',
 'b3741d57-e94f-4afb-851d-33b2e2f5ce82',
 '079166ef-3e3a-4046-a1b3-15ac8ecac97f',
 'e2f94832-19c4-4453-ba5f-bb4a99114af5']

In [ ]:
# testing if the nodes was inserted correctfully
mongo_client.test.rag.find_one({}, {"embedding": 0, "metadata._node_content": 0})

{'_id': ObjectId('661eac5dd770de712246287b'),
 'id': '67e8a0c1-2b70-407c-974f-c7c38e45b9fe',
 'text': 'Paciente queixa-se de dor de cabeça persistente e tontura há dois dias. Não há febre, mas tem dificuldade para dormir.',
 'metadata': {'description': 'Paciente queixa-se de dor de cabeça persistente e tontura há dois dias. Não há febre, mas tem dificuldade para dormir.',
  'date': '12/04/2024 10:15:00',
  'patient_name': 'Marcia Silva',
  'patient_age': 28,
  'sedentary': 'Não',
  'clinic_id': 1,
  'symptoms': 'Dor de cabeça, tontura, insônia',
  'medicine_prescriptions': 'Paracetamol 500mg, Dramin',
  '_node_type': 'TextNode',
  'document_id': 'bf137d69-cfe8-4109-a1ec-dfb907b2be62',
  'doc_id': 'bf137d69-cfe8-4109-a1ec-dfb907b2be62',
  'ref_doc_id': 'bf137d69-cfe8-4109-a1ec-dfb907b2be62'}}

In [ ]:
# creating the chat bot
from llama_index.core.response.notebook_utils import display_response
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.core.vector_stores.types import MetadataFilters, ExactMatchFilter
from llama_index.core.memory import ChatMemoryBuffer

memory = ChatMemoryBuffer.from_defaults(token_limit=1500)
index = VectorStoreIndex.from_vector_store(vector_store)

chat_engine = index.as_chat_engine(
    chat_mode="context",
    filters=MetadataFilters(
        filters=[
            ExactMatchFilter(
                key="metadata.clinic_id",
                value=3,
            )
        ]
    ),
    memory=memory,
    system_prompt=(
        "Você é um chatbot que atuara como assistente de uma clinica"
        " você irá responder sobre qualquer dúvida de forma objetiva em relação aos pacientes da clínica."
    ),
)

In [ ]:
query = "qual foi a última vez que o paciente Filipe Mansano esteve na clínica?"
response = chat_engine.chat(query)
display_response(response)

**`Final Response:`** A última consulta do paciente Filipe Mansano na clínica foi em 26/04/2024 às 15:30.

In [ ]:
query = "e oque foi receitado para ele?"
response = chat_engine.chat(query)
display_response(response)

**`Final Response:`** Na última consulta, foram prescritos os seguintes medicamentos para o paciente Filipe Mansano:
- Dipirona 500mg
- Repouso
- Aumento de líquidos

In [ ]:
query = "ele esteve outros dias na clinica além do dia 12?"
response = chat_engine.chat(query)
display_response(response)

**`Final Response:`** Sim, o paciente Filipe Mansano esteve na clínica em duas ocasiões:
1. No dia 12/04/2024, relatando dores no corpo, febre e náuseas, e foram prescritos os medicamentos Vonal, dipirona e Dorflex.
2. No dia 26/04/2024, relatando fadiga e dores musculares ocasionais, e foram prescritos Dorflex e recomendação para fisioterapia.

In [ ]:
query = "me resume a vida do paciente Filipe Mansano"
response = chat_engine.chat(query)
display_response(response)

**`Final Response:`** O paciente Filipe Mansano, de 31 anos, é sedentário e tem relatado sintomas como fadiga, dores musculares, dores no corpo, febre leve e náuseas em suas consultas na clínica. Ele recebeu prescrições de medicamentos como Dipirona 500mg, Vonal e Dorflex, além de recomendações de repouso, aumento de líquidos e fisioterapia.